In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import pickle
import textwrap
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

from discharge_summaries.openai_llm.chat_models import AzureOpenAIChatModel
from discharge_summaries.schemas.mimic import BHC, PhysicianNote
from discharge_summaries.snomed.lookup import SnomedLookup
from discharge_summaries.snomed.retriever import SnomedRetriever
from discharge_summaries.writers.bhc import BHCWriter

In [ ]:
load_dotenv()

In [ ]:
MIMIC_III_DIR = (
    Path.cwd().parent / "data" / "physionet.org" / "files" / "mimiciii" / "1.4"
)
BHC_FPATH = MIMIC_III_DIR / "BHCS.json"
PHYSICIAN_NOTE_FPATH = MIMIC_III_DIR / "physician_notes_mimic.csv"

SNOMED_DIR = Path.cwd().parent / "data" / "snomed"
TUNED_PHRASE_MATCHER_FPATH = SNOMED_DIR / "tuned_snomed_phrase_matcher.pkl"
PROMPT_MESSAGE_FPATH = Path.cwd() / "prompt_message.txt"
AZURE_ENGINE = "gpt-4"
AZURE_API_VERSION = "2023-07-01-preview"
EXAMPLE_DIR = Path.cwd() / "example_gpt4"

In [ ]:
bhcs = [BHC(**bhc_dict) for bhc_dict in json.loads(BHC_FPATH.read_text())]

In [ ]:
import random

train_set = random.Random(23).sample(bhcs, 100)

In [ ]:
physician_notes = pd.read_csv(PHYSICIAN_NOTE_FPATH)

In [ ]:
sample_bhc = train_set[4]
sample_physician_notes = [
    PhysicianNote(text=row["TEXT"], timestamp=row["CHARTTIME"], hadm_id=row["HADM_ID"])
    for _, row in physician_notes[
        physician_notes["HADM_ID"] == sample_bhc.hadm_id
    ].iterrows()
]

In [ ]:
print(sample_bhc)

In [ ]:
print(sample_bhc.full_text)

In [ ]:
print(sample_physician_notes[0])

### Load Models

In [ ]:
snomed_phrase_matcher = pickle.load(TUNED_PHRASE_MATCHER_FPATH.open("rb"))

In [ ]:
snomed_lookup = SnomedLookup.load(SNOMED_DIR)

In [ ]:
llm = AzureOpenAIChatModel(
    api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=AZURE_API_VERSION,
    engine=AZURE_ENGINE,
    temperature=0,
    timeout=20,
)

In [ ]:
snomed_retriever = SnomedRetriever(
    snomed_phrase_matcher, snomed_lookup, token_window_size=150
)

In [ ]:
# snomed_phrase_matcher._nlp.enable_pipe("parser")

In [ ]:
bhc_writer = BHCWriter(llm, snomed_retriever, train_set[:3], logging_dir=EXAMPLE_DIR)

In [ ]:
bhc = bhc_writer(sample_physician_notes)

In [ ]:
The patient is an 86-year-old male with a significant history of coronary artery disease (CAD), 
status post coronary artery bypass grafting (CABG), and percutaneous coronary intervention (PCI) 
to the left circumflex artery (LCx) in 2154. He presented with a one-hour history of mid-sternal, 
band-like chest pain, rated 4-24 in severity, which was not relieved by nitroglycerin. His medication 
regimen includes Amiodarone, Furosemide, Metoprolol Succinate, Nitroglycerin (as needed), 
Spironolactone, Warfarin, Aspirin, and a short course of Prednisone. The patient's 
CAD is characterized by severe native three-vessel disease, with a totally occluded right 
coronary artery (RCA), a 30% lesion in the left main artery, a 70% proximal lesion and total occlusion 
in the mid portion of the left anterior descending artery (LAD), and diffuse disease in the LCx with a 
90% mid LCx lesion involving the 2nd obtuse marginal. The patient's chest pain is likely related to his 
extensive CAD. His physical examination was unremarkable, with no signs of stasis dermatitis, ulcers, 
scars, or xanthomas, and his pulses were palpable bilaterally. His labs showed a slightly elevated 
creatinine level of 1.5, and an INR of 4.4. The patient's chest pain and CAD should continue to be
managed with his current medication regimen, and any changes in his chest pain should prompt further
 evaluation.

In [ ]:
print(sample_bhc.full_text)

In [ ]:
print(textwrap.fill(bhc.problem_sections[0].text, 80))

In [ ]:
# Sepsis: Upon admission, patient was febrile with leukocytosis,
left shift, lactate 2.8. UA positive as a likely source. CXR
obtained and negative for infiltrate. Additionally, his abdomen
was soft, non-tender, and guaiac negative. He was treated with
IV resuscitation 4L and lactate normalized. Started on
Ceftriaxone 1g q24H with plan to add Vancomycin if clinical
deterioration. He did not require pressors or central line
placement. As BPs improved to > 100mmHg systolic, patient was
transefered to medical floor for further management. He was
noted to have a urine culture positive to Proteus and sensitive
to cephalosporins. He was initially started on Cefpodoxime but
was then changed to cefuroxime as the mirco-organism was
sensitive to this. Per discussion with family, his mental
status returned to baseline by HD#3. He was discharged on 200mg
Hospital1 of cefpodoxime for an additional 7 days, which may be
switched to 250mg Hospital1 of Cefuroxime for an additional 7 days if
permitted by pharmacy supplies. His Blood cultures will require
follow up at Hospital1 18, as they were pending at time of discharge.
He will also require follow up of CBC and LFTs due to start of
the cephalosporin.

In [ ]:
print(sample_bhc.full_text)

In [ ]:
# GIB bleed - most likely due to duodenal ulcer given CT scan.
# GI and surgery were consulted, and given the patient and son's
# desire for conservative management, it was agreed upon that no
# intervention would be performed unless pt developed life
# threatening bleed. Pt received total of 5U PRBCs last on 8-14.
# Her Month/Day (4) was stable at 33-35 on discharge on 8-16. She was
# tolerating a regular pureed diet with supervision given concern
# for aspiration while recovering from UTI. She was discharged
# home on omeprazole twice daily. her aspirin and plavix were
# discontinued. she should discuss restarting her aspirin with
# her primary care physician in the future.

In [ ]:
"""Respiratory failure - First ABG with acidemia and hypercarbia.
Differential includes respiratory infection or mucous plugging
vs. upper airway compromise from malignancy vs. narcotic
overdose given home med use. Report of ?pus with intubation
concerning for pneumonia which was more apparent on repeat CXR
after fluid resuscitation. Suspect aspiration given altered
mental status and location of infiltrate. Sputum culture
pending, but with multiple organisms on gram stain. Patient
initially on vancomycin, zosyn, and levofloxacin to cover for
healthcare associated pneumonia and narrowed to vancomycin and
zosyn (plus metronidazole for C.diff). Urine legionella antigen
was negative. Ventilating very well on 9-27 with 50% FiO2
(tidal volumes in 800s) but extubation held given transfer to
WXVA and still with significant secretions requiring suctioning."""

In [ ]:
for note in sample_physician_notes:
    print(note.text)

In [ ]:
print(sample_physician_notes[2].text)

In [ ]:
print(
    bhc_writer._extract_spans_to_text(
        extract_spans, [note.timestamp for note in sample_physician_notes]
    )
)

In [ ]:
finding_to_extract_text = {
    finding: self._extract_spans_to_text(extract_spans, timestamps)
    for finding, extract_spans in finding_to_extract_spans.items()
    if extract_spans
}

In [ ]:
finding_to_extract_spans

In [ ]:
print(textwrap.fill(bhc.full_text, 80))

In [ ]:
print(sample_bhc.full_text)

In [ ]:
(EXAMPLE_DIR / "gpt_bhc.json").write_text(json.dumps(bhc.dict(), indent=4))

In [ ]:
(EXAMPLE_DIR / "gt.txt").write_text(sample_bhc.full_text)

In [ ]:
paragraphs = [bhc.assessment_and_plan] + [
    f"{problem_section.heading}: {problem_section.text}"
    for problem_section in bhc.problem_sections
]
paragraphs = [textwrap.fill(para, 80) for para in paragraphs]
(EXAMPLE_DIR / "pred.txt").write_text("\n\n".join(paragraphs))

In [ ]:
print(bhc.full_text)